In [ ]:
#Importing required libraries and packages.
import numpy as np
import pandas as pd
import tableprint as tp
from scipy.signal import detrend

### Defining the Mexican hat function based on:
###### http://en.wikipedia.org/wiki/Mexican_hat_wavelet
The A coefficient is added to control the amplitude of the Mexican hat.

In [ ]:
def mexhat(t, A, b):

    return (2 * A /(np.sqrt(3 * b) * np.pi ** 0.25)) * (1 - t ** 2 / b ** 2) * np.exp(-t ** 2 / (2 * b ** 2))

In [ ]:
# Defining a function to find the correlation between wind speed data and Mexican Hat shape
def detect_mexican_hat(A_vals, b_vals, wind_speed):

    # Create empty arrays to store results
    speed_index = []
    match_scores = []
    bs = []
    MH_A = []
    amps = []

    # Loop over values of A and D
    for b in b_vals:

        # Apply the filter to the wind speed data in a rolling window
        for i in range(int(4*b) + start_time_index, end_time_index - start_time_index - int(4*b) + 1):
            wind_speed_window = wind_speed[i +start_time_index - int(4*b):i + start_time_index + int(4*b)]
            detrended_window = detrend(wind_speed_window)

            for A in A_vals:

                # Define a filter to identify Mexican hat events
                mexican_hat_filter = mexhat(np.arange(-4*b, 4*b+1), A, b)

                # Assuming filtered_window and mexican_hat_filter are two arrays with different shapes
                if detrended_window.shape[0] < mexican_hat_filter.shape[0]:
                    detrended_window_repeated = np.repeat(detrended_window[-1], mexican_hat_filter.shape[0])
                    detrended_window_repeated[:detrended_window.shape[0]] = detrended_window
                    mexican_hat_filter_repeated = mexican_hat_filter
                else:
                    detrended_window_repeated = detrended_window
                    mexican_hat_filter_repeated = np.repeat(mexican_hat_filter[-1], detrended_window.shape[0])
                    mexican_hat_filter_repeated[:mexican_hat_filter.shape[0]] = mexican_hat_filter

                window_match_score = np.corrcoef(detrended_window_repeated, mexican_hat_filter_repeated)[0,1]


                max_element = max(detrended_window, key=lambda x: abs(x))
                amp = max_element - detrended_window[0]

                speed_index.append(i)
                match_scores.append(window_match_score)
                bs.append(b)
                MH_A.append(A)
                amps.append(amp)

    return speed_index, match_scores, bs, MH_A, amps

In [ ]:
# Read wind speed data from CSV file
df = pd.read_csv('Sample.csv')
df.head()

In [ ]:
start_time_index  = 0
end_time_index = 10000
wind_speed = df['50m']

# Plot the wind speed data and the detected Mexican hat events
time = np.arange(start_time_index,end_time_index)
time_stamp = df['timestamp']

# create arrays of values for A and b
A_vals = np.arange(1, 30.1, 0.1)
b_vals = np.arange(1, 5.1, 0.1)

# create meshgrid of A and b values
A, b = np.meshgrid(A_vals, b_vals)

# apply detect_mexican_hat function to entire arrays
speed_index, match_scores, bs, MH_A, amps = detect_mexican_hat(A_vals, b_vals, wind_speed)

# flatten the result and convert to list
all_mexican_hats = np.column_stack((speed_index, match_scores, Ds, MH_A, amps)).tolist()

# sort the list by wind speed in descending order
all_mexican_hats.sort(key=lambda tup: tup[0], reverse=False)

#Checking the duplicates and only select the one with the highest score
optimum_results = []
for i in range(len(all_mexican_hats)):
    if i == 0 or (all_mexican_hats[i][0] - all_mexican_hats[i-1][0]) > 0:
        optimum_results.append(all_mexican_hats[i])
    else:
        if abs(all_mexican_hats[i][1]) > abs(optimum_results[-1][1]):
            optimum_results[-1] = all_mexican_hats[i]


optimum_peaks = [tup[0] for tup in optimum_results]
peak_time_stamp = [time_stamp[tup[0]] for tup in optimum_results]
speed_at_peak = [wind_speed[tup[0]] for tup in optimum_results]
optimum_score = [tup[1] for tup in optimum_results]
optimum_b = [tup[2] for tup in optimum_results]
optimum_A = [tup[3] for tup in optimum_results]
optimum_amps = [tup[4] for tup in optimum_results]


data = np.column_stack((optimum_peaks, peak_time_stamp, speed_at_peak, optimum_b, optimum_A, optimum_amps , optimum_score))
headers = ('Time Index', 'Time', 'Wind Speed (m/s)', 'Optimum b', 'Optimum A', 'MH Amplitude', 'Match Score')

In [ ]:
table = tp.table(data,headers, width=20)